In [ ]:
from flask import Flask, render_template, request, send_file, Response
import os
from gtts import gTTS
from moviepy.editor import *
from translate import Translator
import math


#app = Flask(__name__)
app = Flask(__name__)
app.config['STATIC_FOLDER'] = 'static'


# Define a list of Indian languages and their codes
indian_languages = {
    'Hindi': 'hi',
    'Bengali': 'bn',
    'Tamil': 'ta',
    'Telugu': 'te',
    'Marathi': 'mr',
    # Add more languages as needed
}

# Function to handle video generation
def generate_video(text, target_language):
    # Translate the text using the `translate` module
    translator = Translator(to_lang=indian_languages[target_language])
    translated_text = translator.translate(text)

    # Translate the text to audio using gTTS
    tts = gTTS(translated_text, lang=indian_languages[target_language])
    tts.save('generated_audio.mp3')

    # Load a background GIF (make sure 'background.gif' is in the same directory)
    #background_clip = VideoFileClip('Attention.gif')  # Change the filename
    gif_clip = VideoFileClip('static/train.gif')

    # Set the duration of the video to match the background GIF's duration
    gifclip_duration = gif_clip.duration

    # Create a TextClip with the translated text
    txt_clip = TextClip(text, fontsize=30, color='white', bg_color='transparent', size=(300, 400), method='caption')

    # Set the duration (in seconds) for the text clip
    #txt_clip = txt_clip.set_duration(duration)

    # Load the generated audio as an AudioFileClip
    audio_clip = AudioFileClip('generated_audio.mp3')
    
    #audio duration
    txt_clip = txt_clip.set_fps(30)
    audioclip_duration = audio_clip.duration
    
    background_clip = gif_clip.loop(n=math.ceil(audioclip_duration/gifclip_duration))
    txt_clip = txt_clip.set_duration(background_clip.duration)

    # Set the audio of the text clip
    video_clip = txt_clip.set_audio(audio_clip)

    # Position the text in the center of the video
    video_clip = video_clip.set_position("center")

    # Overlay the text on top of the background GIF
    final_clip = CompositeVideoClip([background_clip, video_clip])
    
    # Define the folder path where you want to save the video
    output_folder =  '/Users/adinz/Final_flask/static/'
    
    # Write the video to a file in the specified folder
    # Specify the filename you want for the generated video
    output_filename = f"static/my_language_video.mp4"
    
    #output_filename = f"{output_folder}output_video_{target_language}.mp4"
    #final_clip.write_videofile(output_filename, codec='libx264', fps=24)
    
    # Write the video to the specified filename
    final_clip.write_videofile(output_filename, codec='libx264')

    # Remove the generated video file after it's displayed
    #os.remove('generated_audio.mp3')
    return output_filename

# Route to render the HTML form
@app.route("/")
def upload():
    return render_template('transition.html')

# Route to handle form submission
@app.route("/success", methods=["POST"])
def success():
    text = request.form.get('text_input')
    lang = request.form.get('language_dropdown')

    # Calling the video generation function
    video_path = generate_video(text, lang)
    
    # Return the generated video as a response
    return render_template('transition.html', video_path=video_path)

@app.route('/videos/<filename>')
def display_video(filename):
    video_path = os.path.join('/Users/adinz/Final_flask/static/', filename)
    with open(video_path, "rb") as video_file:
        response = Response(video_file.read(), mimetype="video/mp4")
    return response

if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)
